# (Py)FLAGR

Fuse, Learn, AGgregate, Rerank

FLAGR is a high performing, modular library for rank aggregation. To ensure the highest possible performance, the core FLAGR library is written in C++ and implements a wide collection of unsupervised rank aggregation methods. Its modular design allows third-party programmers to implement their own algorithms and easily rebuild the entire library. FLAGR can be built as a standard application, or as a shared library (`so` or `dll`). In the second case, it can be linked from other C/C++ programs, or even from programs written in other languages (e.g. Python, PHP, etc.).

In this context, PyFLAGR is a Python library that links to FLAGR and allows a developer to exploit the efficient FLAGR implementations from a standard Python program.


## Compiling FLAGR as a shared library

The FLAGR shared library has been already pre-built and tested with the GCC compiler. The FLAGR Github repository includes the appropriate `.so` and `.dll` dynamic libraries in the `pyflagr` directory, so PyFLAGR can be immediately installed without compilation.

Nevertheless, in case a custom rank aggregation method has been implemented with FLAGR, or any modification in the C++ code has been made, FLAGR must be rebuilt and PyFLAGR must be reinstalled. For Linux-based systems with the GCC compiler, FLAGR can be built as a shared library by invoking the following system command:

`g++ -O3 -Wall -Werror -shared -std=c++11 -fPIC /home/leo/Desktop/code/FLAGR/cflagr.cpp -o /home/leo/Desktop/code/FLAGR/pyflagr/flagr.so`

This command will generate the necessary `.so` library.

For Windows-based systems with the GCC compiler, FLAGR can be built as a Dynamic Link Library by invoking the following system commands:

* `g++ -O3 -c -o flagr.o C:/Users/Leo/Documents/cpp_algorithms/FLAGR/cflagr.cpp`
* `g++ -O3 -o C:/Users/Leo/Documents/cpp_algorithms/FLAGR/pyflagr/pyflagr/flagr.dll -s -shared flagr.o -Wl,--subsystem,windows`

- OR -
* `g++ -O3 -c -o /home/leo/Desktop/code/FLAGR/pyflagr/pyflagr/flagr.o /home/leo/Desktop/code/FLAGR/cflagr.cpp`
* ``
This command will generate the necessary `.dll` library.


## Installing PyFLAGR

The installation of PyFLAGR can be performed by employing the `pip` Python package installer. Initially we need to navigate to the directory where the `setup.py` file resides.

In the current Linux Mint installation the directory is `cd /home/leo/Desktop/code/FLAGR/pyflagr/`

In the current Windows7 installation the directory is `cd C:\Users\Leo\Documents\cpp_algorithms\FLAGR\pyflagr`

After navigating to the directory where `setup.py` resides, install it by using `pip`:

`pip install .`

## Importing and using PyFLAGR

PyFLAGR groups its supported rank aggregation methods in four modules:

1. `Comb`: In this module the `CombSUM` and `CombMNZ` methods are implemented. Each method comes in four variants according to the rank/score normalization method. Future releases of FLAGR will also include CombAVG, CombMAX and CombMIN.
2. `Majoritarian`: Includes `CondorcetWinners` and `OutrankingApproach`.
3. `MarkovChains`: The fourth and most popular method (termed `MC4`) based on Markov Chains is implemented. Future releases of FLAGR will include the other three implementations.
4. `Weighted`: This module implements several self-weighting rank aggregation methods. These methods automatically identify the expert voters and include:
 1. The Preference Relations Graph method of .
 2. The Agglomerative method of .
 3. The Iterative, Distance-Based method of .

The following statements demonstrate the imports of all PyFLAGR rank aggregation methods in a typical jupyter notebook.

In [1]:
import pyflagr.Comb as SCORE_BASED
import pyflagr.Majoritarian as ORDER_BASED
import pyflagr.MarkovChains as MARKOV_CHAINS
import pyflagr.Weighted as WGT


All PyFLAGR rank aggregation methods include:
* a standard class constructor: several hyper-parameters of the corresponding algorithm  and other execution arguments can be passed through the constructor. All the constructor inputs have default values, therefore, they are considered optional. This means that all constructors can be called *any* argument at all.
* an `aggregate` method that runs the algorithm on the selected input and (optionally) evaluates the generated aggregate list. In all algorithms, `aggregate` method accepts the following arguments:

| Parameter    | Type                                         | Default Value  | Values  |
| :----------- | :--------------------------------------------| :--------------| :------ |
| `input_file` | String - Required, unless `input_df` is set. | Empty String   | A CSV file that contains the input lists to be aggregated. |
| `input_df` | Pandas DataFrame - Required, unless `input_file` is set. | `None` | A Pandas DataFrame that contains the input lists to be aggregated. **Note:** If both `input_file` and `input_df` are set, only the former is used; the latter is ignored. |
| `rels_file`  | String, Optional. | Empty String | A CSV file that contains the relevance judgements of the involved list elements. If such a file is passed, FLAGR will evaluate the generated aggregate list/s by computing several retrieval effectiveness evaluation measures. The results of the evaluation will be stored in the `eval_df` DataFrame. Otherwise, no evaluation will take place and `eval_df` will be empty. Read more on the evaluation of rank aggregation quality. |
| `rels_df`    | Pandas DataFrame, Optional. | `None` | A Pandas DataFrame that contains the relevance judgements of the involved list elements. If such a dataframe is passed, FLAGR will evaluate the generated aggregate list/s by computing several retrieval effectiveness evaluation measures. The results of the evaluation will be stored in the `eval_df` DataFrame. Otherwise, no evaluation will take place and `eval_df` will be empty. Read more on the evaluation of rank aggregation quality. **Note:** If both `rels_file` and `rels_df` are set, only the former is used; the latter is ignored. |
| `output_dir` | String, Optional. | Temporary directory (OS-specific) | The directory where the output files (aggregate lists and evaluation) will be stored. If it is not set, the default location will be used. |


## Input data

The core library, FLAGR, accepts data (namely, the input lists to be aggregated) in a single, specially formatted CSV file. The columns in the CSV file are organized according to the following manner:

`Query/Topic,Voter,Item,Score,Algorithm/Dataset`

where:
* `Query/Topic`: the query string or the topic for which the list is submitted.
* `Voter`: the name of the voter, or the ranker who submits the list.
* `Item`: a unique name that identifies a particular element in the list. A voter cannot submit the same element for the same query/topic two or more times. This means that each element appears exactly once in each list. However, the same element may appear in lists submitted by other voters.
* `Score`: the score assigned to an `Item` by a specific `Voter`. In may cases (e.g. search engine rankings), the individual scores are unknown. In such cases the scores can be replaced by the (reverse) ranking of an `Item` in such a manner that the top rankings receive higher scores than the ones that have been assigned lower rankings.

PyFLAGR has two mechanisms for passing data to FLAGR, namely:
* either by forwarding the name and the location of the aforementioned input CSV file (this is the `input_file` argument of the `aggregate` method),
* or by accepting a Pandas Dataframe from the user (this is the `input_df` argument of the `aggregate` method). In this case, PyFLAGR internally dumps the `input_df` contents into a temporary CSV file and passes the name and the location of that temporary file to FLAGR.


Optionally, the user may specify a second CSV file (called as `rels_file`), or a Dataframe (called as `rels_df`) that contain judgments about the relevance of the included elements w.r.t a query. The columns in `rels_file` are organized as follows:

`Query/Topic,0,Item,Relevance`

where:
* `Query/Topic`: the query string or the topic for which the corresponding `Item` is evaluated.
* `0`: A hypothetical hyper-voter (also called voter `0`) who has flawless knowledge of the `Query/Topic` and determines whether an `Item` is relevant to it, or not. The value of this column must be always `0`.
* `Item`: a unique name that identifies a particular element of which the relevance to the `Query/Topic` is evaluated.
* `Relevance`: the relevance score assigned to an `Item` by `Voter 0`.


## Output data format

In all cases the core library, FLAGR, writes the generated aggregate list in a plain CSV file. PyFLAGR reads the contents of this CSV file into a Pandas Dataframe which is returned to the user.

Optionally, FLAGR may also create a second output file to write the results of the evaluation of the effectiveness of an algorithm. This happens when a `rels_file` is provided to the algorithm. The `aggregate` method of all algorithms *always* returns two Pandas Dataframes according to the provided input.


## Code examples

The following examples demonstrate the usage of all PyFLAGR rank aggregation methods.

In [2]:
#lists = '/media/leo/B65266EC5266B1331/phd_Research/08 - Datasets/TREC/Synthetic/MOSO.csv'
#qrels = '/media/leo/B65266EC5266B1331/phd_Research/08 - Datasets/TREC/Synthetic/MOSO_qrels.csv'

lists = 'D:/phd_Research/08 - Datasets/TREC/Synthetic/MOSO.csv'
qrels = 'D:/phd_Research/08 - Datasets/TREC/Synthetic/MOSO_qrels.csv'


### Comb methods: CombSUM

Member of `pyflagr.Comb`.

The `CombSUM` constructor supports the following parameters:


| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `norm` | String, Optional. | `borda` | Rank or score normalization methods:<ul><li>`borda`: The aggregation is performed by normalizing the element *rankings* according to the Borda normalization method. Equivalent to the `BordaCount` function.</li><li>`rank`: The aggregation is performed by normalizing the element *rankings* according to the Rank normalization method.</li><li>`score`: The aggregation is performed by normalizing the element *scores* according to the Score normalization method.</li><li>`z-score`: The aggregation is performed by normalizing the element *scores* according to the Z-Score normalization method.</li></ul> |


In [3]:
csum = SCORE_BASED.CombSUM(norm='rank')

# In this case, rels_file has been specified, so PyFLAGR returns two non-blank dataframes:
# * df_out contains the aggregate list produced by the selected algorithm
# * df_eval contains the effectiveness evaluation based on the relevance judgments in the rels_file
df_out, df_eval = csum.aggregate(input_file=lists, rels_file=qrels)

df_out.head(10)

FileNotFoundError: Could not find module 'C:\Users\Leo\Documents\cpp_algorithms\FLAGR\pyflagr\pyflagr\flagr.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
csum = SCORE_BASED.CombSUM(norm='rank')

# In this case, rels_file has NOT been specified, so PyFLAGR returns two dataframes,
# * df_out contains the aggregate list produced by the selected algorithm
# * df_eval is blank
df_out, df_eval = csum.aggregate(input_file=lists)

df_out.head(10)


### Comb methods: BordaCount

Member of `pyflagr.Comb`.

`BordaCount` is equivalent to `CombSUM` with `borda` normalization. Its constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |


In [ ]:
borda = SCORE_BASED.BordaCount()

df_out, df_eval = borda.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


In [ ]:
# Equivalent code for Borda Count: This one produces the same results as the previous code block
csum = SCORE_BASED.CombSUM(norm='borda')

df_out, df_eval = csum.aggregate(input_file=lists, rels_file=qrels)

df_eval.tail(1)

### Comb methods: CombMNZ

Member of `pyflagr.Comb`.

The `CombMNZ` constructor supports the following parameters:


| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `norm` | String, Optional. | `borda` | Rank or score normalization methods:<ul><li>`borda`: The aggregation is performed by normalizing the element *rankings* according to the Borda normalization method. Equivalent to the `BordaCount` function.</li><li>`rank`: The aggregation is performed by normalizing the element *rankings* according to the Rank normalization method.</li><li>`score`: The aggregation is performed by normalizing the element *scores* according to the Score normalization method.</li><li>`z-score`: The aggregation is performed by normalizing the element *scores* according to the Z-Score normalization method.</li></ul> |


In [ ]:
cmnz = SCORE_BASED.CombMNZ(norm='rank')

df_out, df_eval = cmnz.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Majoritarian methods: Concorcet Winners

Member of `pyflagr.Majoritarian`.

The `CondorcetWinners` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |


In [ ]:
condorcet = ORDER_BASED.CondorcetWinners()

df_out, df_eval = condorcet.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Majoritarian methods: Copeland Winners

Member of `pyflagr.Majoritarian`.

The `CopelandWinners` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |


In [ ]:
copeland = ORDER_BASED.CopelandWinners()

df_out, df_eval = copeland.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Majoritarian methods: Outranking Approach

Member of `pyflagr.Majoritarian`.

The `OutrankingApproach` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `pref` | Hyperparameter, Float, Optional. | 0    | Preference threshold.  |
| `veto` | Hyperparameter, Float, Optional. | 0.75 | Veto threshold.        |
| `conc` | Hyperparameter, Float, Optional. | 0    | Concordance threshold. |
| `disc` | Hyperparameter, Float, Optional. | 0.25 | Discordance threshold. |


In [ ]:
outrank = ORDER_BASED.OutrankingApproach()

df_out, df_eval = outrank.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Markov Chain methods: MC4

Member of `pyflagr.MarkovChains`.

The `MC4` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `ergodic_number` | Hyperparameter, Float, Optional. | 0.15    | The ergodic number.  |
| `delta` | Hyperparameter, Float, Optional. | 0.00000001 | The $\delta$ hyperparameter.        |
| `max_iterations` | Hyperparameter, Integer, Optional. | 200    | Maximum number of iterations. |


In [ ]:
mch4 = MARKOV_CHAINS.MC4()

df_out, df_eval = mch4.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Weighted methods: Preferelence Relations Graph

Member of `pyflagr.Weighted`.

The `PreferenceRelationsGraph` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `alpha`| Hyperparameter, Float, Optional. | 0.1 | The $\alpha$ hyper-parameter of the algorithm.  |
| `beta` | Hyperparameter, Float, Optional. | 0.5 | The $\beta$ hyper-parameter of the algorithm.  |


In [ ]:
prf_graph = WGT.PreferenceRelationsGraph(alpha=0.1, beta=0.5)

df_out, df_eval = prf_graph.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


### Weighted methods: Agglomerative Aggregation

Member of `pyflagr.Weighted`.

The `Agglomerative` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `c1` | Hyperparameter, Float, Optional. | 2.5 | The $c_1$ hyper-parameter of the algorithm.  |
| `c2` | Hyperparameter, Float, Optional. | 1.5 | The $c_2$ hyper-parameter of the algorithm.  |

In [ ]:
agg = WGT.Agglomerative(c1=0.1, c2=0.5)



### Weighted methods: Iterative Distance-Based Aggregation

Member of `pyflagr.Weighted`.

The `DIBRA` constructor supports the following parameters:

| Parameter      | Type        | Default Value  | Values |
| :------------- | :---------- | :--------------| :------|
| `eval_pts` | Integer, Optional. Considered only if `rels_file` or `rels_df` is set. | 10 | Determines the elements in the aggregate list on which the evaluation measures (i.e. Precision, and nDCG) will be computed. For example, for `eval_pts=10` FLAGR will compute $P@1, P@2, ... P@10$, and $N@1, N@2, ..., N@10$. |
| `aggregator` | Hyperparameter, String, Optional. | `combsum:borda` | The baseline aggregation method. An extended weighted variant of the baseline method is applied internally by plugging the computed voter weights.<br> The list of the supported values includes:<br><ul><li>`combsum:borda`: CombSUM with Borda rank normalization.</li><li>`combsum:rank`: CombSUM with rankings normalization.</li><li>`combsum:score`: CombSUM with score min-max normalization.</li><li>`combsum:z-score`: CombSUM with score z-normalization.</li><li>`combmnz:borda`: CombMNZ with Borda rank normalization.</li><li>`combmnz:rank`: CombMNZ with rankings normalization.</li><li>`combmnz:score`: CombMNZ with score min-max normalization.</li><li>`combmnz:z-score`: CombMNZ with score z-normalization.</li><li>`condorcet`: The Condorcet Winners method.</li><li>`outrank`: The Outranking Approach.</li></ul>|
| `w_norm` | Hyperparameter, String, Optional. | `minmax` | The voter weights normalization method. The list of the supported values includes:<br><ul><li>`none`: The voter weights will not be normalized.</li><li>`minmax`: The voter weights will be normalized with min-max scaling.</li><li>`z`: The voter weights will be z-normalized</li></ul> |
| `dist` | Hyperparameter, String, Optional. | `cosine` | The metric that is used to measure the distance between an input list and the temporary aggregate list. The list of the supported values includes:<br><ul><li>`rho`: The Spearman's $\rho$ correlation coefficient.</li><li>`cosine`: Cosine similarity of the lists' vector representations.</li><li>`tau`: The Kendall's $\tau$ correlation coefficient.</li><li>`footrule`: A scaled variant of Spearman's Footrule distance.</li></ul> |
| `gamma` | Hyperparameter, Float, Optional. | 1.50 | Regulates the weight convergence speed. |
| `prune` | Hyperparameter, Boolean, Optional. | `False` | Triggers a weight-dependant list pruning mechanism. |
| `d1`    | Hyperparameter, Float, Optional. Used only when `prune=True` | 0.4 | The hyperparameter $\delta_1$ of the weight-dependant list pruning mechanism. |
| `d2`    | Hyperparameter, Float, Optional. Used only when `prune=True` | 0.1 | The hyperparameter $\delta_2$ of the weight-dependant list pruning mechanism. |
| `tol`    | Hyperparameter, Float, Optional. | 0.01 | Controls the convergence precision. This tolerance threshold represents the minimum precision of the difference between the voter weight in an iteration and the voter weight of the previous iteration.|
| `max_iter` | Hyperparameter, Integer, Optional. | 50 | Controls the maximum number of iterations. FLAGR will stop the execution of DIBRA if the requested number of iterations have been performed, even if the voter weights have not fully converged.|
| `pref` | Hyperparameter, Float, Optional. | 0    | Preference threshold.  |
| `veto` | Hyperparameter, Float, Optional. | 0.75 | Veto threshold.        |
| `conc` | Hyperparameter, Float, Optional. | 0    | Concordance threshold. |
| `disc` | Hyperparameter, Float, Optional. | 0.25 | Discordance threshold. |


In [ ]:
method_1 = WGT.DIBRA()

df_out, df_eval = method_1.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


In [ ]:
method_2 = WGT.DIBRA(eval_pts=20, gamma=1.5, prune=True, d1=0.3, d2=0.05)

df_out, df_eval = method_2.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)


In [ ]:
method_3 = WGT.DIBRA(eval_pts=20, aggregator="outrank")

df_out, df_eval = method_3.aggregate(input_file=lists, rels_file=qrels)
df_eval.tail(1)
